In [59]:
!pip install imbalanced-learn
!pip install tensorflow
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2
!pip install scikeras
!pip install keras  # or the version that fits your needs

Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2
  Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.3 MB)


In [60]:
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import numpy as np

In [61]:
data = pd.read_csv('/content/Tweets.csv')
data.head(3)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)


In [62]:
data= data[['airline_sentiment','text']]
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [63]:
data.dropna(axis=0, inplace=True)

<ipython-input-63-1edeb318a43a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(axis=0, inplace=True)


In [64]:
data.shape

(14640, 2)

In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   airline_sentiment  14640 non-null  object
 1   text               14640 non-null  object
dtypes: object(2)
memory usage: 228.9+ KB


In [66]:
data.describe()

,airline_sentiment,text
count,14640,14640
unique,3,14427
top,negative,@united thanks
freq,9178,6


In [67]:
data.isnull().sum()

,0
airline_sentiment,0
text,0


In [68]:
data['airline_sentiment'] = data['airline_sentiment'].map({'positive': 1, 'negative': 0, 'neutral':2})
data.sample(4)

<ipython-input-68-f9d8a1b5630e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['airline_sentiment'] = data['airline_sentiment'].map({'positive': 1, 'negative': 0, 'neutral':2})


,airline_sentiment,text
13541,0,@AmericanAir @emrey35 But it says your agents ...
2829,2,@united need to add service dog to my itinerar...
802,0,"@united tag numbers 0016 964012, 0016 964077, ..."
3753,1,@united Thanks - it's very helpful to understa...


In [69]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [70]:
def clean_text(text):
    # 1. Menghapus tanda baca (punctuation)
    text_no_punctuation = re.sub(r'[^\w\s]', '', text)

    # 2. Menghapus angka
    text_no_numbers = re.sub(r'\d+', '', text_no_punctuation)

    # 3. Menghapus stopword
    stop_words = set(stopwords.words('indonesian'))
    words = text_no_numbers.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Menggabungkan kembali teks
    return ' '.join(filtered_words)

data['text'] = data['text'].apply(clean_text)


In [71]:
data.head()

,airline_sentiment,text
0,2,VirginAmerica What dhepburn said
1,1,VirginAmerica plus youve added commercials to ...
2,2,VirginAmerica I didnt today Must mean I need t...
3,0,VirginAmerica its really aggressive to blast o...
4,0,VirginAmerica and its a really big bad thing a...


In [72]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_NB_WORD = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORD, filters  = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(data['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Found 14564 unique tokens.
Shape of data tensor: (14640, 250)


In [73]:
datakategori= data['airline_sentiment'].value_counts()
print(datakategori)

airline_sentiment
0    9178
2    3099
1    2363
Name: count, dtype: int64


In [74]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Memisahkan fitur dan label
y = data['airline_sentiment']

# Cek distribusi kelas sebelum undersampling
print("Distribusi kelas sebelum undersampling:", Counter(y))

# Membuat objek RandomUnderSampler
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Melakukan undersampling
X_res, y_res = undersampler.fit_resample(X, y)

# Cek distribusi kelas setelah undersampling
print("Distribusi kelas setelah undersampling:", Counter(y_res))


Distribusi kelas sebelum undersampling: Counter({0: 9178, 2: 3099, 1: 2363})
Distribusi kelas setelah undersampling: Counter({0: 2363, 1: 2363, 2: 2363})


In [75]:
import numpy as np

# Hitung jumlah data per kelas
print("Jumlah data per kelas sebelum oversampling:", np.bincount(y))
print("Jumlah data per kelas setelah oversampling:", np.bincount(y_res))


Jumlah data per kelas sebelum oversampling: [9178 2363 3099]
Jumlah data per kelas setelah oversampling: [2363 2363 2363]


In [76]:
Y = pd.get_dummies(y_res).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (7089, 3)


In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res,Y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5671, 250) (5671, 3)
(1418, 250) (1418, 3)


In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(Embedding(MAX_NB_WORD, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(150, dropout=0.3, recurrent_dropout=0.2, kernel_regularizer=l2(0.01)))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape=(None, X.shape[1]))
print(model.summary())

# model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 250, 100)            │       5,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_4                  │ (None, 250, 100)            │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 150)                 │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │             453 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,151,053 (19.65 MB)

 Trainable params: 5,151,053 (19.65 MB)

 Non-trainable params: 0 (0.00 B)

None


In [79]:
from keras.callbacks import EarlyStopping
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 36s 487ms/step - accuracy: 0.4356 - loss: 2.1100 - val_accuracy: 0.5260 - val_loss: 1.0672 - learning_rate: 0.0010
Epoch 2/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 33s 455ms/step - accuracy: 0.5876 - loss: 0.9861 - val_accuracy: 0.6247 - val_loss: 0.8839 - learning_rate: 0.0010
Epoch 3/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 40s 448ms/step - accuracy: 0.7092 - loss: 0.7654 - val_accuracy: 0.7093 - val_loss: 0.7721 - learning_rate: 0.0010
Epoch 4/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 41s 442ms/step - accuracy: 0.7963 - loss: 0.6015 - val_accuracy: 0.7066 - val_loss: 0.7785 - learning_rate: 0.0010
Epoch 5/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 32s 457ms/step - accuracy: 0.8599 - loss: 0.4637 - val_accuracy: 0.7198 - val_loss: 0.7592 - learning_rate: 0.0010
Epoch 6/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 43s 488ms/step - accuracy: 0.8940 - loss: 0.3823 - val_accuracy: 0.7207 - val_loss: 0.7817 - learning_rate: 0.0010
Epoch 7/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 39s 465ms/step - accuracy: 0.9145 - l

In [80]:
y_pred = model.predict(X_test)
y_pred.shape

45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step


(1418, 3)

In [81]:
from sklearn.metrics import classification_report

# Mendapatkan prediksi dari model
y_pred = model.predict(X_test)

# Jika menggunakan one-hot encoding, ubah prediksi ke label kelas integer
y_pred_classes = y_pred.argmax(axis=-1)
y_test_classes = y_test.argmax(axis=-1)

# Cetak classification report
print(classification_report(y_test_classes, y_pred_classes))


45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 123ms/step
              precision    recall  f1-score   support

           0       0.77      0.74      0.75       469
           1       0.75      0.78      0.76       472
           2       0.69      0.70      0.70       477

    accuracy                           0.74      1418
   macro avg       0.74      0.74      0.74      1418
weighted avg       0.74      0.74      0.74      1418



In [82]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, SpatialDropout1D

model2 = Sequential()
model2.add(Embedding(MAX_NB_WORD, EMBEDDING_DIM))
model2.add(SpatialDropout1D(0.2))
model2.add(SimpleRNN(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(3, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.build(input_shape=(None, X.shape[1]))
print(model2.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 250, 100)            │       5,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_5                  │ (None, 250, 100)            │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             303 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,020,403 (19.15 MB)

 Trainable params: 5,020,403 (19.15 MB)

 Non-trainable params: 0 (0.00 B)

None


In [86]:
from keras.callbacks import EarlyStopping
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)
model2.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Epoch 1/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.9086 - loss: 0.2808 - val_accuracy: 0.5577 - val_loss: 1.1037 - learning_rate: 0.0010
Epoch 2/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9115 - loss: 0.2497 - val_accuracy: 0.5515 - val_loss: 1.0903 - learning_rate: 0.0010
Epoch 3/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9234 - loss: 0.2368 - val_accuracy: 0.5656 - val_loss: 1.1864 - learning_rate: 0.0010
Epoch 4/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9332 - loss: 0.1899 - val_accuracy: 0.5815 - val_loss: 1.2206 - learning_rate: 0.0010
Epoch 5/100
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - accuracy: 0.9451 - loss: 0.1607 - val_accuracy: 0.5753 - val_loss: 1.2987 - learning_rate: 0.0010


In [87]:
y_pred2 = model2.predict(X_test)
y_pred2.shape

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


(1418, 3)

In [89]:
from sklearn.metrics import classification_report

# Mendapatkan prediksi dari model
y_pred = model2.predict(X_test)

# Jika menggunakan one-hot encoding, ubah prediksi ke label kelas integer
y_pred_classes = y_pred.argmax(axis=-1)
y_test_classes = y_test.argmax(axis=-1)

# Cetak classification report
print(classification_report(y_test_classes, y_pred_classes))


45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
              precision    recall  f1-score   support

           0       0.58      0.51      0.55       469
           1       0.59      0.59      0.59       472
           2       0.47      0.52      0.50       477

    accuracy                           0.54      1418
   macro avg       0.55      0.54      0.54      1418
weighted avg       0.55      0.54      0.54      1418

